In [1]:
import tensorflow as tf
import numpy as np
import cv2
import input_data

In [2]:
img_height=200
img_width=100
#Network Parameters
n_input=img_height*img_width*3
learning_rate=5e-5   #1e-5
training_iters=10000
batch_size= 4
display_step=5
dropout=0.9
epoch=25


#tf graph input
x=tf.placeholder(tf.float32,[None,img_height,img_width,3])
y=tf.placeholder(tf.float32,[None,img_height,img_width,16])
keep_prob=tf.placeholder(tf.float32)

In [3]:
def conv2d(img,w,b):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(img,w,strides=[1,1,1,1],padding='SAME'),b))
def max_pool(img,k):
    return tf.nn.max_pool(img,ksize=[1,k,k,1],strides=[1,k,k,1],padding='SAME')
def conv2d_transpose(img,w,b,output_shape):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d_transpose(img,w,output_shape=output_shape,strides=[1,1,1,1],padding='SAME'),b))
def cv_bottleneck(img,w1,w2,w3):
    tmp_bn_1=tf.nn.relu(tf.nn.conv2d(img,w1,strides=[1,1,1,1],padding='SAME'))
    tmp_bn_2=tf.nn.relu(tf.nn.conv2d(tmp_bn_1,w2,strides=[1,1,1,1],padding='SAME'))
    tmp_bn_3=tf.nn.relu(tf.nn.conv2d(tmp_bn_2,w3,strides=[1,1,1,1],padding='SAME'))
    return (img+tmp_bn_3)
def dc_bottleneck(img,w1,w2,output_shape):
    output_shape[3]=w1.get_shape().as_list()[2]
    tmp_bn_1=tf.nn.relu(tf.nn.conv2d_transpose(img,w1,output_shape=output_shape,strides=[1,1,1,1],padding='SAME'))
    output_shape[3]=w2.get_shape().as_list()[2]
    tmp_bn_2=tf.nn.conv2d_transpose(tmp_bn_1,w2,output_shape=output_shape,strides=[1,1,1,1],padding='SAME')
    return (img+tmp_bn_2)
def hourglass(img,wc1_1,wc1_2,wc1_3,bc1,wc2_1,wc2_2,wc2_3,bc2,wc3_1,wc3_2,wc3_3,bc3,wc4_1,wc4_2,wc4_3,bc4,wc5_1,wc5_2,wc5_3,bc5):
    tmp_cv_1=tf.nn.relu(tf.nn.bias_add(cv_bottleneck(img,wc1_1,wc1_2,wc1_3),bc1))
    tmp_cv_1_pooling=max_pool(tmp_cv_1,k=2)
    tmp_cv_1_pooling=tf.nn.dropout(tmp_cv_1_pooling,keep_prob)
    
    tmp_cv_2=tf.nn.relu(tf.nn.bias_add(cv_bottleneck(tmp_cv_1_pooling,wc2_1,wc2_2,wc2_3),bc2))
    tmp_cv_2_pooling=max_pool(tmp_cv_2,k=2)
    tmp_cv_2_pooling=tf.nn.dropout(tmp_cv_2_pooling,keep_prob)
    
    tmp_cv_3=tf.nn.relu(tf.nn.bias_add(cv_bottleneck(tmp_cv_2_pooling,wc3_1,wc3_2,wc3_3),bc3))
    tmp_cv_3=tf.nn.dropout(tmp_cv_3,keep_prob)
    
    tmp_rs_1=tf.image.resize_bilinear(tmp_cv_3,tmp_cv_2.get_shape().as_list()[1:3])
    #tmpShape=tmp_cv_1_pooling.get_shape().as_list()
    #tmpShape[0]=batch_size
    #tmp_dc_1=tf.nn.relu(tf.nn.bias_add(dc_bottleneck(tmp_rs_1,wc4_1,wc4_2,output_shape=tmpShape),bc4))
    tmp_dc_1=tf.nn.relu(tf.nn.bias_add(cv_bottleneck(tmp_rs_1,wc4_1,wc4_2,wc4_3),bc4))
    
    tmp_add_1=tf.add(tmp_dc_1,tmp_cv_1_pooling)
    tmp_add_1=tf.nn.dropout(tmp_add_1,keep_prob)
    
    tmp_rs_2=tf.image.resize_bilinear(tmp_add_1,tmp_cv_1.get_shape().as_list()[1:3])
    #tmpShape=img.get_shape().as_list()
    #tmpShape[0]=batch_size
    #tmp_dc_2=tf.nn.relu(tf.nn.bias_add(dc_bottleneck(tmp_rs_2,wc5_1,wc5_2,output_shape=tmpShape),bc5))
    tmp_dc_2=tf.nn.relu(tf.nn.bias_add(cv_bottleneck(tmp_rs_2,wc5_1,wc5_2,wc5_3),bc5))
    
    tmp_add_2=tf.add(tmp_dc_2,img)
    tmp_add_21=tf.nn.dropout(tmp_add_2,keep_prob)
    
    return (tmp_add_2)

In [4]:
wc0=tf.Variable(tf.random_normal([15,15,3,64]),name="wc0")
bc0=tf.Variable(tf.random_normal([64]),name="bc0")

wc1=tf.Variable(tf.random_normal([8,8,64,128]),name="wc1")
bc1=tf.Variable(tf.random_normal([128]),name="bc1")

wh1_1_1=tf.Variable(tf.random_normal([5,5,128,64]),name="wh1_1_1")
wh1_1_2=tf.Variable(tf.random_normal([5,5,64,64]),name="wh1_1_2")
wh1_1_3=tf.Variable(tf.random_normal([5,5,64,128]),name="wh1_1_3")

bh1_1=tf.Variable(tf.random_normal([128]),name="bh1_1")

wh1_2_1=tf.Variable(tf.random_normal([3,3,128,64]),name="wh1_2_1")
wh1_2_2=tf.Variable(tf.random_normal([3,3,64,64]),name="wh1_2_2")
wh1_2_3=tf.Variable(tf.random_normal([3,3,64,128]),name="wh1_2_3")

bh1_2=tf.Variable(tf.random_normal([128]),name="bh1_2")

wh1_3_1=tf.Variable(tf.random_normal([3,3,128,64]),name="wh1_3_1")
wh1_3_2=tf.Variable(tf.random_normal([3,3,64,64]),name="wh1_3_2")
wh1_3_3=tf.Variable(tf.random_normal([3,3,64,128]),name="wh1_3_3")


bh1_3=tf.Variable(tf.random_normal([128]),name="bh1_3")

wh1_4_1=tf.Variable(tf.random_normal([3,3,128,64]),name="wh1_4_1")
wh1_4_2=tf.Variable(tf.random_normal([3,3,64,64]),name="wh1_4_2")
wh1_4_3=tf.Variable(tf.random_normal([3,3,64,128]),name="wh1_4_3")

bh1_4=tf.Variable(tf.random_normal([128]),name="bh1_4")

wh1_5_1=tf.Variable(tf.random_normal([3,3,128,64]),name="wh1_5_1")
wh1_5_2=tf.Variable(tf.random_normal([3,3,64,64]),name="wh1_5_2")
wh1_5_3=tf.Variable(tf.random_normal([3,3,64,128]),name="wh1_5_3")

bh1_5=tf.Variable(tf.random_normal([128]),name="bh1_5")

wd1=tf.Variable(tf.random_normal([7,7,64,128]),name="wd1")
bd1=tf.Variable(tf.random_normal([64]),name="bd1")

wd2=tf.Variable(tf.random_normal([5,5,32,64]),name="wd2")
bd2=tf.Variable(tf.random_normal([32]),name="bd2")

wc2=tf.Variable(tf.random_normal([3,3,32,16]),name="wc2")
bc2=tf.Variable(tf.random_normal([16]),name="bc2")

wc3=tf.Variable(tf.random_normal([3,3,16,16]),name="wc3")
bc3=tf.Variable(tf.random_normal([16]),name="bc3")

In [5]:
#Construct model
x=x/255.0
conv0=conv2d(x,wc0,bc0)
conv0_pooling=max_pool(conv0,k=2)
conv0_pooling=tf.nn.dropout(conv0_pooling,keep_prob)

conv1=conv2d(conv0_pooling,wc1,bc1)
conv1_pooling=max_pool(conv1,k=2)
conv1_pooling=tf.nn.dropout(conv1_pooling,keep_prob)

hg_1=hourglass(conv1_pooling,wh1_1_1,wh1_1_2,wh1_1_3,bh1_1,wh1_2_1,wh1_2_2,wh1_2_3,bh1_2,wh1_3_1,wh1_3_2,wh1_3_3,bh1_3,wh1_4_1,wh1_4_2,wh1_4_3,bh1_4,wh1_5_1,wh1_5_2,wh1_5_3,bh1_5)

resize1=tf.image.resize_bilinear(hg_1,conv1.get_shape().as_list()[1:3])
tmpShape=conv0_pooling.get_shape().as_list()
tmpShape[0]=batch_size
tmpShape[3]=wd1.get_shape().as_list()[2]
dconv1=conv2d_transpose(resize1,wd1,bd1,output_shape=tmpShape)
dconv1=tf.nn.dropout(dconv1,keep_prob)

resize2=tf.image.resize_bilinear(dconv1,conv0.get_shape().as_list()[1:3])
tmpShape=x.get_shape().as_list()
tmpShape[0]=batch_size
tmpShape[3]=wd2.get_shape().as_list()[2]
dconv2=conv2d_transpose(resize2,wd2,bd2,output_shape=tmpShape)
dconv2=tf.nn.dropout(dconv2,keep_prob)

conv2=conv2d(dconv2,wc2,bc2)
conv2=tf.nn.dropout(conv2,keep_prob)

conv3=conv2d(conv2,wc3,bc3)
conv3=tf.nn.dropout(conv3,keep_prob)

#cost=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=conv2,labels=y))
#cost=tf.reduce_mean(tf.abs(tf.subtract(conv2,y)))
loss_h=[]
'''for index_heat in range(0,16):
    for index_height in range(0,img_height):
        for index_width in range(0,img_width):
            b_2=tf.log(tf.add(tf.subtract(1.0,conv3[:,index_height,index_width,index_heat]),1e-15))
            b_1=tf.subtract(1.0,y[:,index_height,index_width,index_heat])
            b=tf.multiply(b_1,b_2)
            
            a_2=tf.log(tf.add(conv3[:,index_height,index_width,index_heat],1e-15))
            checka=tf.check_numerics(a_2,"s_2")
            a_1=y[:,index_height,index_width,index_heat]
            a=tf.multiply(a_1,a_2)
            
            c=tf.add(a,b)
            
            loss_h.append(tf.reduce_sum(c))'''

for tmp in range(0,16):
    b_2=tf.log(tf.add(tf.abs(tf.subtract(1.0,conv3[:,:,:,tmp])),0.00000001))
    checkb=tf.check_numerics(b_2,"b_2")
    b_1=tf.subtract(1.0,y[:,:,:,tmp])
    b=tf.multiply(b_1,b_2)
    
    a_2=tf.log(tf.add(tf.abs(conv3[:,:,:,tmp]),0.00000001))
    checka=tf.check_numerics(a_2,"s_2")
    a_1=y[:,:,:,tmp]
    a=tf.multiply(a_1,a_2)
    c=tf.add(a,b)
    loss_h.append(tf.reduce_sum(c))
    

    #tmp_one=tf.reduce_mean(tf.abs(tf.subtract(y[:,:,:,tmp],conv3[:,:,:,tmp])))
    #check=tf.check_numerics(tmp_one,"tmp_one is nan or inf")
    #loss_h.append(tmp_one)
cost=tf.reduce_mean(loss_h)/batch_size
#optimizer=tf.train.RMSPropOptimizer(learning_rate=learning_rate.minimize(cost)
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate,epsilon=1e-8).minimize(loss=cost,global_step=tf.train.get_global_step())

In [6]:
'''saver = tf.train.Saver({"wc0":wc0,"bc0":bc0,"wc1":wc1,"bc1":bc1,
                        "wh1_1_1":wh1_1_1,"wh1_1_2":wh1_1_2,"wh1_1_3":wh1_1_3,"bh1_1":bh1_1,
                        "wh1_2_1":wh1_2_1,"wh1_2_2":wh1_2_2,"wh1_2_3":wh1_2_3,"bh1_2":bh1_2,
                        "wh1_3_1":wh1_3_1,"wh1_3_2":wh1_3_2,"wh1_3_3":wh1_3_3,"bh1_3":bh1_3,
                        "wh1_4_1":wh1_4_1,"wh1_4_2":wh1_4_2,"wh1_4_3":wh1_4_3,"bh1_4":bh1_4,
                        "wh1_5_1":wh1_5_1,"wh1_5_2":wh1_5_2,"wh1_5_3":wh1_5_3,"bh1_5":bh1_5,
                        "wd1":wd1,"bd1":bd1,"wd2":wd2,"bd2":bd2,"wc2":wc2,"bc2":bc2,"wc3":wc3,"bc3":bc3
                       },max_to_keep=20)'''
saver=tf.train.Saver(max_to_keep=25)
init=tf.global_variables_initializer()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [7]:
with tf.Session(config=config) as sess:
    sess.run(init)
    #saver.restore(sess,"model/model_hg_15.ckpt")
    #saver = tf.train.Saver()
    #saver.restore(sess,"model/model_hg_14_6.ckpt")
    for i in range(0,epoch):
        print("epoch:"+str(i)+"       !!!!!!!!!!!!")
        step=0
        now_at=0
        while step*batch_size<training_iters:
            X=input_data.x_next_batch(
                    img_dir_path='mpii_human_pose_v1\\output_images100x200\\',
                    index_path='train_data\\new_data_100x200.json',
                    img_height=img_height,img_width=img_width,
                    batch_size=batch_size,now_at=now_at)
            Y=input_data.y_next_batch(
                    img_dir_path='mpii_human_pose_v1\\output_images100x200\\',
                    index_path='train_data\\new_data_100x200.json',
                    img_height=img_height,img_width=img_width,
                    batch_size=batch_size,now_at=now_at)
            sess.run(optimizer,feed_dict = {x:X,y:Y,keep_prob:dropout})
            #msg=sess.run([checka],feed_dict = {x:X,y:Y,keep_prob:dropout})
            #print(msg[0])
            #print(msg[1])
            loss=sess.run(cost,feed_dict = {x:X,y:Y,keep_prob:1.})
            if step%display_step==0:
                print("Iter "+str(step*batch_size)+", Minibatch Loss="+"{:.6f}".format(loss))
            step+=1
            now_at+=batch_size
        save_path = saver.save(sess, "model/model_hg_neg_"+str(i)+".ckpt")
print("done!")

epoch:0       !!!!!!!!!!!!
Iter 0, Minibatch Loss=850747.500000
Iter 20, Minibatch Loss=815894.625000
Iter 40, Minibatch Loss=791454.312500
Iter 60, Minibatch Loss=760559.937500
Iter 80, Minibatch Loss=750565.875000
Iter 100, Minibatch Loss=728215.375000
Iter 120, Minibatch Loss=713416.625000
Iter 140, Minibatch Loss=708197.500000
Iter 160, Minibatch Loss=708229.062500
Iter 180, Minibatch Loss=677225.750000
Iter 200, Minibatch Loss=671877.000000
Iter 220, Minibatch Loss=643819.937500
Iter 240, Minibatch Loss=615222.875000
Iter 260, Minibatch Loss=579453.750000
Iter 280, Minibatch Loss=546514.625000
Iter 300, Minibatch Loss=525412.625000
Iter 320, Minibatch Loss=509096.000000
Iter 340, Minibatch Loss=481526.718750
Iter 360, Minibatch Loss=472724.093750
Iter 380, Minibatch Loss=455671.250000
Iter 400, Minibatch Loss=438284.500000
Iter 420, Minibatch Loss=424241.187500
Iter 440, Minibatch Loss=409432.687500
Iter 460, Minibatch Loss=396215.843750
Iter 480, Minibatch Loss=392190.937500
Iter

Iter 4120, Minibatch Loss=105441.992188
Iter 4140, Minibatch Loss=105429.359375
Iter 4160, Minibatch Loss=105416.968750
Iter 4180, Minibatch Loss=105403.554688
Iter 4200, Minibatch Loss=105419.031250
Iter 4220, Minibatch Loss=105403.390625
Iter 4240, Minibatch Loss=105395.328125
Iter 4260, Minibatch Loss=105400.250000
Iter 4280, Minibatch Loss=105374.671875
Iter 4300, Minibatch Loss=105364.093750
Iter 4320, Minibatch Loss=105366.781250
Iter 4340, Minibatch Loss=105355.718750
Iter 4360, Minibatch Loss=105334.703125
Iter 4380, Minibatch Loss=105324.640625
Iter 4400, Minibatch Loss=105313.117188
Iter 4420, Minibatch Loss=105299.671875
Iter 4440, Minibatch Loss=105285.054688
Iter 4460, Minibatch Loss=105279.617188
Iter 4480, Minibatch Loss=105262.453125
Iter 4500, Minibatch Loss=105251.601562
Iter 4520, Minibatch Loss=105240.375000
Iter 4540, Minibatch Loss=105227.281250
Iter 4560, Minibatch Loss=105218.515625
Iter 4580, Minibatch Loss=105212.093750
Iter 4600, Minibatch Loss=105204.609375


Iter 8220, Minibatch Loss=104049.390625
Iter 8240, Minibatch Loss=104043.265625
Iter 8260, Minibatch Loss=104038.187500
Iter 8280, Minibatch Loss=104032.296875
Iter 8300, Minibatch Loss=104024.734375
Iter 8320, Minibatch Loss=104018.390625
Iter 8340, Minibatch Loss=104012.078125
Iter 8360, Minibatch Loss=104005.187500
Iter 8380, Minibatch Loss=103997.804688
Iter 8400, Minibatch Loss=103992.250000
Iter 8420, Minibatch Loss=103986.359375
Iter 8440, Minibatch Loss=104071.375000
Iter 8460, Minibatch Loss=103975.695312
Iter 8480, Minibatch Loss=103969.609375
Iter 8500, Minibatch Loss=103962.554688
Iter 8520, Minibatch Loss=103957.765625
Iter 8540, Minibatch Loss=103953.781250
Iter 8560, Minibatch Loss=103951.031250
Iter 8580, Minibatch Loss=103945.648438
Iter 8600, Minibatch Loss=103940.187500
Iter 8620, Minibatch Loss=103928.890625
Iter 8640, Minibatch Loss=103917.593750
Iter 8660, Minibatch Loss=103908.328125
Iter 8680, Minibatch Loss=103899.734375
Iter 8700, Minibatch Loss=103900.562500


Iter 2340, Minibatch Loss=103389.218750
Iter 2360, Minibatch Loss=103388.281250
Iter 2380, Minibatch Loss=103386.609375
Iter 2400, Minibatch Loss=103383.000000
Iter 2420, Minibatch Loss=103378.382812
Iter 2440, Minibatch Loss=103373.218750
Iter 2460, Minibatch Loss=103368.726562
Iter 2480, Minibatch Loss=103362.328125
Iter 2500, Minibatch Loss=103356.281250
Iter 2520, Minibatch Loss=103348.671875
Iter 2540, Minibatch Loss=103340.906250
Iter 2560, Minibatch Loss=103334.429688
Iter 2580, Minibatch Loss=103327.117188
Iter 2600, Minibatch Loss=103320.156250
Iter 2620, Minibatch Loss=103312.937500
Iter 2640, Minibatch Loss=103311.140625
Iter 2660, Minibatch Loss=103310.710938
Iter 2680, Minibatch Loss=103309.687500
Iter 2700, Minibatch Loss=103305.148438
Iter 2720, Minibatch Loss=103300.281250
Iter 2740, Minibatch Loss=103295.609375
Iter 2760, Minibatch Loss=103290.898438
Iter 2780, Minibatch Loss=103286.671875
Iter 2800, Minibatch Loss=103283.039062
Iter 2820, Minibatch Loss=103277.546875


Iter 6440, Minibatch Loss=101784.453125
Iter 6460, Minibatch Loss=101779.867188
Iter 6480, Minibatch Loss=101775.593750
Iter 6500, Minibatch Loss=101774.171875
Iter 6520, Minibatch Loss=101774.109375
Iter 6540, Minibatch Loss=101769.703125
Iter 6560, Minibatch Loss=101764.781250
Iter 6580, Minibatch Loss=101759.000000
Iter 6600, Minibatch Loss=101752.484375
Iter 6620, Minibatch Loss=101746.523438
Iter 6640, Minibatch Loss=101740.375000
Iter 6660, Minibatch Loss=101734.812500
Iter 6680, Minibatch Loss=101717.382812
Iter 6700, Minibatch Loss=101700.328125
Iter 6720, Minibatch Loss=101684.625000
Iter 6740, Minibatch Loss=101669.203125
Iter 6760, Minibatch Loss=101653.203125
Iter 6780, Minibatch Loss=101637.375000
Iter 6800, Minibatch Loss=101625.429688
Iter 6820, Minibatch Loss=101615.515625
Iter 6840, Minibatch Loss=101653.109375
Iter 6860, Minibatch Loss=101604.171875
Iter 6880, Minibatch Loss=101598.585938
Iter 6900, Minibatch Loss=101594.781250
Iter 6920, Minibatch Loss=101591.289062


...

Iter 1340, Minibatch Loss=39504.515625
Iter 1360, Minibatch Loss=39489.093750
Iter 1380, Minibatch Loss=39487.960938
Iter 1400, Minibatch Loss=39477.601562
Iter 1420, Minibatch Loss=39453.207031
Iter 1440, Minibatch Loss=39423.781250
Iter 1460, Minibatch Loss=39444.773438
Iter 1480, Minibatch Loss=39430.734375
Iter 1500, Minibatch Loss=39425.765625
Iter 1520, Minibatch Loss=39443.703125
Iter 1540, Minibatch Loss=39414.191406
Iter 1560, Minibatch Loss=39405.941406
Iter 1580, Minibatch Loss=39398.226562
Iter 1600, Minibatch Loss=39394.726562
Iter 1620, Minibatch Loss=39388.875000
Iter 1640, Minibatch Loss=39362.710938
Iter 1660, Minibatch Loss=39346.765625
Iter 1680, Minibatch Loss=39346.984375
Iter 1700, Minibatch Loss=39341.312500
Iter 1720, Minibatch Loss=39334.398438
Iter 1740, Minibatch Loss=39327.933594
Iter 1760, Minibatch Loss=39299.937500
Iter 1780, Minibatch Loss=39226.773438
Iter 1800, Minibatch Loss=39191.953125
Iter 1820, Minibatch Loss=39170.082031
Iter 1840, Minibatch Loss

Iter 5560, Minibatch Loss=39479.332031
Iter 5580, Minibatch Loss=39478.007812
Iter 5600, Minibatch Loss=39475.234375
Iter 5620, Minibatch Loss=39471.144531
Iter 5640, Minibatch Loss=39472.792969
Iter 5660, Minibatch Loss=39472.898438
Iter 5680, Minibatch Loss=39451.332031
Iter 5700, Minibatch Loss=39434.335938
Iter 5720, Minibatch Loss=39468.257812
Iter 5740, Minibatch Loss=39464.328125
Iter 5760, Minibatch Loss=39467.515625
Iter 5780, Minibatch Loss=39467.500000
Iter 5800, Minibatch Loss=39466.289062
Iter 5820, Minibatch Loss=39435.656250
Iter 5840, Minibatch Loss=39421.097656
Iter 5860, Minibatch Loss=39410.246094
Iter 5880, Minibatch Loss=39393.906250
Iter 5900, Minibatch Loss=39394.375000
Iter 5920, Minibatch Loss=39407.492188
Iter 5940, Minibatch Loss=39412.812500
Iter 5960, Minibatch Loss=39402.582031
Iter 5980, Minibatch Loss=39457.640625
Iter 6000, Minibatch Loss=39447.238281
Iter 6020, Minibatch Loss=39420.605469
Iter 6040, Minibatch Loss=39412.531250
Iter 6060, Minibatch Loss

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\users\gatl\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-3a5304c2e558>", line 25, in <module>
    loss=sess.run(cost,feed_dict = {x:X,y:Y,keep_prob:1.})
  File "c:\users\gatl\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\client\session.py", line 900, in run
    run_metadata_ptr)
  File "c:\users\gatl\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\client\session.py", line 1135, in _run
    feed_dict_tensor, options, run_metadata)
  File "c:\users\gatl\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\client\session.py", line 1316, in _do_run
    run_metadata)
  File "c:\users\gatl\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\client\session.py", line 1322, in _do_call
    return fn(*args

KeyboardInterrupt: 